# Challenge 2 - Deep Learning

In this tutorial, you will train a PyTorch model on the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset using distributed training via Nccl/Gloo across a GPU cluster. 

## 1. Import Azure ML Python Python SDK

In [ ]:
import azureml.core
print("SDK version:", azureml.core.VERSION)

## 2. Authentication and initializing Azure Machine Learning Workspace

As a first step you have to authenticate against the Azure [Machine Learning Workspace](https://ml.azure.com/). This can be achieved in different ways:

1. **Interactive Login Authentication:** The interactive authentication is suitable for local experimentation on your own computer.
2. **Azure CLI Authentication:** Azure CLI authentication is suitable if you are already using Azure CLI for managing Azure resources, and want to sign in only once.
3. **Managed Service Identity (MSI) Authentication:** The MSI authentication is suitable for automated workflows, for example as part of Azure Devops build.
4. **Service Principal Authentication:** The Service Principal authentication is suitable for automated workflows, for example as part of Azure Devops build.

For now, we will use the interactive authentication, which is the default mode when using Azure ML SDK. When you connect to your workspace using `Workspace.from_config`, you will get an interactive login dialog.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()

In [ ]:
print("Workspace name: " + ws.name, 
      "Azure region: " + ws.location, 
      "Subscription id: " + ws.subscription_id, 
      "Resource group: " + ws.resource_group, sep = '\n')

## 3. Create Compute Engine

In this sample, we want to train a simple scikit-learn model on a remote compute engine on Azure. To do so, we first must create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target).

In this challenge, we want to use Azure ML managed compute ([AmlCompute](https://docs.microsoft.com/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute)) for our remote training compute resource. Once this is created, you are ready to train on your remote compute.

#### **Task:** Create a machine learning compute target.

Create an Azure Machine Learning Compute cluster and folow the steps one to four.
1. Check whether the cluster with the given name already exists.
2. Create the configuration (this step is local and only takes a second). Use the SKU `STANDARD_NC6` and a maximum of 4 nodes.
3. Create the cluster (this step will take about 20 seconds)
4. Provision the VMs to bring the cluster to the initial size. This step will take about 3-5 minutes and is providing only sparse output in the process. Please make sure to wait until the call returns before moving to the next cell.

**TASK**: Create a GPU cluster with the VM SKU as given above

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = ???

    # create the cluster
    compute_target = ComputeTarget.create(???, ???, ???)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

## 4. Create a project directory 

Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script and any additional files your training script depends on.

In [ ]:
TRAIN_FOLDER_NAME = 'train'
TRAIN_FILE_NAME = 'train.py'

In [ ]:
import os
os.makedirs(os.path.join(".", TRAIN_FOLDER_NAME), exist_ok=True)

## 5. Create a training script 

Now you will need to create your training scripts in your project folder. This will be done in the next step. In practice, you should be able to take any custom training script as is and run it with Azure ML without having to modify your code.

If you would like to use Azure ML's [tracking and metrics](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#metrics) capabilities, you will have to add a small amount of Azure ML code inside your training script.

In `train_iris.py`, we will log some metrics to our Azure ML run. To do so, we will access the Azure ML Run object within the script:

```python
from azureml.core.run import Run
run = Run.get_context()
```

Further within `train_iris.py`, we log the kernel and penalty parameters, and the highest accuracy the model achieves:

```python
run.log('Kernel type', np.string(args.kernel))
run.log('Penalty', np.float(args.penalty))

run.log('Accuracy', np.float(accuracy))
```

These run metrics will become particularly important when we begin hyperparameter tuning our model in the "Tune model hyperparameters" section.

**TASK**: Fill out the missing values below

In [ ]:
%%writefile $TRAIN_FOLDER_NAME/$TRAIN_FILE_NAME

from __future__ import print_function
import argparse
import os
import shutil
import time
import onnx
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.utils.data
import torch.utils.data.distributed
import torchvision.models as models

from azureml.core.run import Run
# get the Azure ML run object
run = Run.???

# Training settings
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('-j', '--workers', default=4, type=int, metavar='N',
                    help='number of data loading workers (default: 4)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
parser.add_argument('--weight-decay', '--wd', default=1e-4, type=float,
                    metavar='W', help='weight decay (default: 1e-4)')
parser.add_argument('--world-size', default=1, type=int,
                    help='number of distributed processes')
parser.add_argument('--dist-url', type=str,
                    help='url used to set up distributed training')
parser.add_argument('--dist-backend', default='nccl', type=str,
                    help='distributed backend')
parser.add_argument('--rank', default=-1, type=int,
                    help='rank of the worker')

best_prec1 = 0
args = parser.parse_args()

args.distributed = args.world_size >= 2

if args.distributed:
    dist.init_process_group(backend=args.dist_backend, init_method=args.dist_url,
                            world_size=args.world_size, rank=args.rank)

train_dataset = datasets.MNIST('data-%d' % args.rank, train=True, download=True,
                               transform=transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.1307,), (0.3081,))
                               ]))

if args.distributed:
    train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
else:
    train_sampler = None

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=args.batch_size, shuffle=(train_sampler is None),
    num_workers=args.workers, pin_memory=True, sampler=train_sampler)


test_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=args.batch_size, shuffle=False,
    num_workers=args.workers, pin_memory=True)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim = 1)


model = Net()

if not args.distributed:
    model = torch.nn.DataParallel(model).cuda()
else:
    model.cuda()
    model = torch.nn.parallel.DistributedDataParallel(model)

# define loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss().cuda()

optimizer = torch.optim.SGD(model.parameters(), args.lr, momentum=args.momentum, weight_decay=args.weight_decay)


def train(epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to train mode
    model.train()
    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        try:
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
            losses.update(loss.item(), input.size(0))
            top1.update(prec1[0], input.size(0))
            top5.update(prec5[0], input.size(0))

            # compute gradient and do SGD step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % 5 == 0:
                # logging the metrics to the run
                ???("loss", losses.avg)
                ???("prec@1", "{0:.3f}".format(top1.avg))
                ???("prec@5", "{0:.3f}".format(top5.avg))
                print('Epoch: [{0}][{1}/{2}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                      'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(epoch, i, len(train_loader),
                                                                      batch_time=batch_time, data_time=data_time,
                                                                      loss=losses, top1=top1, top5=top5))
        except:
            import sys
            print("Unexpected error:", sys.exc_info()[0])


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


for epoch in range(1, args.epochs + 1):
    train(epoch)
    
# Create 'outputs' folder
os.makedirs(os.path.join(".", 'outputs'), exist_ok=True)

# Save model as pt file
torch.save(model, os.path.join('outputs', 'model.pt'))

# Save model as ONNX file
dummy_input = torch.randn(1, 1, 28, 28, device='cuda')
torch.onnx.export(model,
                  dummy_input,
                  os.path.join('outputs', 'model.onnx'),
                  export_params=True,
                  opset_version=10,
                  do_constant_folding=True,
                  verbose=True,
                  input_names=[ "image" ],
                  output_names=[ "log_softmax_pred" ])

# Check ONNX model
onnx_model = onnx.load(os.path.join('outputs', 'model.onnx'))
onnx.checker.check_model(onnx_model)

## 6. Create an experiment

An *Experiment* is a logical container in an Azure ML Workspace that represents a collection of trials (individual model runs). It hosts run records which can include run metrics and output artifacts from your experiments.

**TASK**: Create new new experiment with the name `pytorch_sample`

In [ ]:
from azureml.core import Experiment
exp = ???

## 7. Create Estimator

An estimator object is used to submit the run. Azure Machine Learning has pre-configured estimators for common machine learning frameworks, as well as generic Estimator. Create a generic estimator for by specifying

- The name of the estimator object, est
- The directory that contains your scripts. All the files in this directory are uploaded into the cluster nodes for execution.
- The training script name
- The input Dataset for training
- The compute target. In this case you will use the AmlCompute you created
- The environment definition for the experiment

**TASK**: Fill in the missing values

In [ ]:
from azureml.train.dnn import PyTorch, Nccl

script_params = {'--dist-backend' : 'nccl',
                 '--dist-url': '$AZ_BATCHAI_PYTORCH_INIT_METHOD',
                 '--rank': '$AZ_BATCHAI_TASK_INDEX',
                 '--world-size': 2,
                 '--epochs': 1}

est = PyTorch(source_directory=???,
              entry_script=???,
              script_params=???,
              compute_target=???,
              node_count=2,
              distributed_training=Nccl(),
              use_gpu=True,
              framework_version='1.3',
              pip_packages=['onnx', 'Pillow==6.1'])

In the above code, `script_params` uses Azure ML generated `AZ_BATCHAI_PYTORCH_INIT_METHOD` for shared file-system initialization and `AZ_BATCHAI_TASK_INDEX` as rank of each worker process.
The above code specifies that we will run our training script on `2` nodes, with one worker per node. In order to execute a distributed run using Nccl, you must provide the argument `distributed_training=Nccl()`. Using this estimator with these settings, PyTorch and dependencies will be installed for you. However, if your script also uses other packages, make sure to install them via the `PyTorch` constructor's `pip_packages` or `conda_packages` parameters.

In [ ]:
'''
# Alternative
from azureml.train.dnn import PyTorch, Gloo

script_params = {'--dist-backend' : 'gloo',
                 '--dist-url': '$AZ_BATCHAI_PYTORCH_INIT_METHOD',
                 '--rank': '$AZ_BATCHAI_TASK_INDEX',
                 '--world-size': 2}

est = PyTorch(source_directory=???,
                    entry_script=???,
                    script_params=???,
                    compute_target=???,
                    node_count=2,
                    distributed_training=Gloo(),
                    use_gpu=True)
'''

In the above code, `script_params` uses Azure ML generated `AZ_BATCHAI_PYTORCH_INIT_METHOD` for shared file-system initialization and `AZ_BATCHAI_TASK_INDEX` as rank of each worker process.
The above code specifies that we will run our training script on `2` nodes, with one worker per node. In order to execute a distributed run using Gloo, you must provide the argument `distributed_training=Gloo()`. Using this estimator with these settings, PyTorch and dependencies will be installed for you. However, if your script also uses other packages, make sure to install them via the `PyTorch` constructor's `pip_packages` or `conda_packages` parameters.

Once you create the estimaotr you can follow the submit steps as shown above to submit a PyTorch run with `Gloo` backend. 

## 8. Submit the job

Submit the estimator to the Azure ML experiment to kick off the execution.

**TASK**: Fill in the missing values to submit the experiment and wait for its completion while the outputs are shown in the notebook

In [ ]:
run = exp.???
run.???

In [ ]:
#run.cancel()

You now have a model trained on a remote cluster. Retrieve all the metrics logged during the run, including the accuracy of the model:

In [ ]:
run.get_metrics()

## 9. Tune model hyperparameters

Now that we've seen how to do a pyTorch training run using the SDK, let's see if we can further improve the accuracy of our model. We can optimize our model's hyperparameters using Azure Machine Learning's hyperparameter tuning capabilities.

Let's tune the `lr` (learning rate), `momentum` and `weight-decay` parameters. In this example we will use random sampling to try different configuration sets of hyperparameters to minimize our primary metric, `loss`.

**TASK**: Fill in the missing values

In [ ]:
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import BayesianParameterSampling, RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import uniform, loguniform
from azureml.train.hyperdrive.policy import BanditPolicy

param_sampling = RandomParameterSampling({
    '--dist-backend' : 'nccl',
    '--dist-url': '$AZ_BATCHAI_PYTORCH_INIT_METHOD',
    '--rank': '$AZ_BATCHAI_TASK_INDEX',
    '--world-size': 2,
    '--lr': loguniform(0.0005, 0.01),
    '--momentum': uniform(0.45, 0.55),
    '--weight-decay': uniform(1e-5, 1e-3)
    })

hyperdrive_run_config = HyperDriveConfig(???=???,
                                         hyperparameter_sampling=???, 
                                         primary_metric_name='???',
                                         primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
                                         max_total_runs=16,
                                         max_concurrent_runs=4,
                                         policy=BanditPolicy(slack_factor=0.1, evaluation_interval=1, delay_evaluation=1))

Finally, lauch the hyperparameter tuning job.

**TASK**: Fill in the missing values to submit the experiment and wait for its completion while the outputs are shown in the notebook

In [ ]:
hyperdrive_run = ???
hyperdrive_run.???

Often times, finding the best hyperparameter values for your model can be an iterative process, needing multiple tuning runs that learn from previous hyperparameter tuning runs. Reusing knowledge from these previous runs will accelerate the hyperparameter tuning process, thereby reducing the cost of tuning the model and will potentially improve the primary metric of the resulting model. When warm starting a hyperparameter tuning experiment with Bayesian sampling, trials from the previous run will be used as prior knowledge to intelligently pick new samples, so as to improve the primary metric. Additionally, when using Random or Grid sampling, any early termination decisions will leverage metrics from the previous runs to determine poorly performing training runs. 

Azure Machine Learning allows you to warm start your hyperparameter tuning run by leveraging knowledge from up to 5 previously completed hyperparameter tuning parent runs. 

Additionally, there might be occasions when individual training runs of a hyperparameter tuning experiment are cancelled due to budget constraints or fail due to other reasons. It is now possible to resume such individual training runs from the last checkpoint (assuming your training script handles checkpoints). Resuming an individual training run will use the same hyperparameter configuration and mount the storage used for that run. The training script should accept the "--resume-from" argument, which contains the checkpoint or model files from which to resume the training run. You can also resume individual runs as part of an experiment that spends additional budget on hyperparameter tuning. Any additional budget, after resuming the specified training runs is used for exploring additional configurations.

For more information on warm starting and resuming hyperparameter tuning runs, please refer to the [Hyperparameter Tuning for Azure Machine Learning documentation](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters) 

When all jobs finish, we can find out the one that has the highest accuracy.

**TASK**: Fill in the missing values to get the best run and print the runDefintion arguments

In [ ]:
best_run = ???
print(best_run.get_details()['???']['???'])

## 10. Register model

The last step in the training script wrote the file `model.pkl` in a directory named `outputs` in the VM of the cluster where the job is executed. `outputs` is a special directory in that all content in this  directory is automatically uploaded to your workspace.  This content appears in the run record in the experiment under your workspace. Hence, the model file is now also available in your workspace.

You can see files associated with that run.

In [ ]:
best_run.get_file_names()

Register the model in the workspace so that you (or other collaborators) can later query, examine, and deploy this model.

**TASK**: Fill in the missing values to register the best model. Remember what model framework we used above to save our model in.

In [ ]:
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model = best_run.???(model_name='pytorch-model',
                                model_path='???/???.???',
                                ???=Model.Framework.???,
                                model_framework_version='1.0',
                                description='PyTorch MNIST classification.',
                                tags={'area': 'mnist', 'type': 'pytorch'})

print(model.name, model.id, model.version, sep='\n')

Now, your model is ready for deployment.

## 11. Deployment

No-code model deployment is currently in preview and supports various frameworks and model types including Tensorflow SavedModel format, ONNX models and Scikit-learn models. No code model deployment is supported for all built-in scikit-learn model types.

The deployment will take a few minutes and will take place on an Azure Container Instance.

**TASK**: Fill in the missing values to create a new web service for our new model and wait until the service creation is completed

In [ ]:
service_no_code = Model.???
service_no_code.???(show_output=True)

In [ ]:
# If deployment fails, then retry with:
# service_no_code.update(models=[model])

In [ ]:
service_no_code.get_logs()

Convert this Webservice object into a JSON serialized dictionary, which lists all the details of the webservice.

In [ ]:
service_no_code.serialize()

## 12. Test Service

The following code is an example of a Python client that can be used with the container.

**TASK**: Fill in the missing values to call the web service with the generated sample data

In [ ]:
import json
import torch

# Two sets of data to score, so we get two results back.
dummy_input = torch.randn(1, 1, 28, 28, device='cpu')
data = {'data':
        [
            dummy_input.tolist()
        ]
        }
# Convert to JSON string.
input_data = ???

# Make the request and display the response.
resp = ???
print(resp)

Delete the service to save cost.

**TASK**: Delete the service again

In [ ]:
???